# RecordList representation capabilities

In [1]:
# Import Client and Session classes
from openerp_proxy.ext.all import HField, HTMLTable
from openerp_proxy import (Client,
                           Session)

cl = Client('localhost', 'openerp_proxy_test_db', 'admin', 'admin', protocol='xml-rpc')
cl

login,admin
Host,localhost
Protocol,xml-rpc
Port,8069
Database,openerp_proxy_test_db


In [2]:
so_list = cl['sale.order'].search_records([])
so_list

Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Object,Object ('sale.order')
Record count,8


## Display simple sale orders in table view

In [3]:
# Use anyfield library to simplify code: https://pypi.python.org/pypi/anyfield
from anyfield import F

# High light rows by condition
highlighters = {
    '#99FF99': F.state == 'done',
    '#9999FF': F.state == 'draft',
    '#FFFF99': F.state == 'progress',
}

# Display as table.
# Note that prefetch method is used to fetch some set of fields with less RPC call.
# on big datasets it may speed up performance signifiantly.
# Each RecordList instance have related cache, which reduce need of reading data on each field get.
so_list.prefetch('id', 'name', 'partner_id', 'partner_id.email', 'state')
so_table = so_list.as_html_table(
    'id',
    'name',
    # _name attribute provides result of *name_search method:
    HField('partner_id._name', name='Partner name'),
    # silent=True means, if field cannot be found, not throw error
    HField('partner_id.email', name='Partner email', silent=True),
    # Also it is posible to display result of method calls
    # 'as_html_list()' is method of RecordList.
    HField(F.order_line.as_html_list(), 'Order lines'),
    'state',
    highlighters=highlighters,
)
so_table

id,name,Partner name,Partner email,Order lines,state
8,SO008,Millennium Industries,False,"20: Laptop Customized21: Mouse, Wireless",draft
7,SO007,Luminous Technologies,False,16: Laptop E502317: GrapWorks Software18: Datacard19: USB Adapter,manual
6,SO006,Think Big Systems,info@thinkbig.com,15: PC Assamble + 2GB RAM,draft
5,SO005,Agrolait,info@agrolait.com,"12: External Hard disk13: Blank DVD-RW14: Printer, All-in-one",draft
4,SO004,Millennium Industries,False,"8: Service on demand9: Webcam10: Multimedia Speakers11: Switch, 24 ports",draft
3,SO003,Chamber Works,info@chamberworks.com,6: On Site Monitoring7: Toner Cartridge,draft
2,SO002,Bank Wealthy and sons,email@wealthyandsons.com,4: Service on demand5: On Site Assistance,draft
1,SO001,Agrolait,info@agrolait.com,"1: Laptop E50232: Pen drive, 16GB3: Headset USB",sent


## Find all partners and display table with partner names and their sale orders

In [4]:
partners = cl._res_partner() # Find all partners
partners_tbl = partners.as_html_table(
    (F._name, 'Name'),
    HField(F.country_id._name, 'Country', silent=True),  # silent is used to ignore exceptions
                                                         # raised while getting field
    
    # anyfield F expressions suports basic logic operations
    HField((F.sale_order_ids & 
            F.sale_order_ids.as_html_table('id',
                                          '_name',
                                          'date_order',
                                          'amount_total',
                                          'state') | 
            False),
           'Sale orders'),
)  
partners_tbl

Name                                    Country         Sale orders
--------------------------------------  --------------  --------------------------------------------------------------
Administrator                                           False
Agrolait                                Belgium         <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2c8d7d0>
Agrolait, Michel Fletcher               Belgium         False
Agrolait, Thomas Passot                 Belgium         False
ASUSTeK                                 Taiwan          False
ASUSTeK, Joseph Walters                 Taiwan          False
ASUSTeK, Tang Tsui                      Taiwan          False
Axelor                                  France          False
Axelor, Laith Jubair                    France          False
Bank Wealthy and sons                   United Kingdom  <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2d3d490>
Bank Wealthy and sons, Charlie Bernard  United Kingdom  False
Bank Wealthy and sons, Jessica Dupont   United Kingdom  False
Best Designers                          India           False
Best Designers, Ayaan Agarwal           India           False
Camptocamp                              France          False
Camptocamp, Luc Maurer                  France          False
Chamber Works                           United States   <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2d3d090>
Chamber Works, Angel Cook               United States   False
Chamber Works, Robert Anderson          United States   False
China Export                            China           False
China Export, Chao Wang                 China           False
China Export, Zhi Ch'ang                China           False
Delta PC                                United States   False
Delta PC, Brian Williams                United States   False
Delta PC, Paul Williams                 United States   False
Delta PC, Richard Ellis                 United States   False
Demo Portal User                                        False
Demo User                                               False
Elec Import                             United States   False
Elec Import, James Miller               United States   False
Epic Technologies                       United States   False
Epic Technologies, David Simpson        United States   False
Epic Technologies, John M. Brown        United States   False
Global Solutions                        United Kingdom  False
Global Solutions, Kevin Clarke          United Kingdom  False
Global Solutions, Morgan Rose           United Kingdom  False
Global Solutions, Robin Smith           United Kingdom  False
Luminous Technologies                   Spain           <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2d3dd10>
Luminous Technologies, Laura Castro     Spain           False
Luminous Technologies, Sergio Pérez     Spain           False
Mediapole                               Germany         False
Mediapole, Phillipp Miller              Germany         False
Millennium Industries                   United Kingdom  <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2c8da50>
Millennium Industries, Jacob Taylor     United Kingdom  False
Nebula Business                         Argentina       False
Nebula Business, Benjamin Flores        Argentina       False
Seagate                                 United States   False
Seagate, Edward Foster                  United States   False
Spark Systems                           Brazil          False
Spark Systems, Arthur Gomez             Brazil          False
Spark Systems, Julia Rivero             Brazil          False
Template User                                           False
The Jackson Group                       United States   False
The Jackson Group, Daniel Jackson       United States   False
The Jackson Group, William Thomas       United States   False
Think Big Systems                       United States   <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2da6150>

In such way HTML Table will be displayed in console

In [5]:
from IPython.display import display_pretty
display_pretty(partners_tbl)

Name                                    Country         Sale orders
--------------------------------------  --------------  --------------------------------------------------------------
Administrator                                           False
Agrolait                                Belgium         <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2c8dc10>
Agrolait, Michel Fletcher               Belgium         False
Agrolait, Thomas Passot                 Belgium         False
ASUSTeK                                 Taiwan          False
ASUSTeK, Joseph Walters                 Taiwan          False
ASUSTeK, Tang Tsui                      Taiwan          False
Axelor                                  France          False
Axelor, Laith Jubair                    France          False
Bank Wealthy and sons                   United Kingdom  <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2c8db90>
Bank Wealthy and sons, Charlie Bernard  United Kingdom  False
Bank Wealthy and 

## Nested tables representation

In [6]:
partners = cl._res_partner() # Find all partners
partners_tbl = partners.as_html_table(
    (F._name, 'Name'),
    HField(F.country_id._name, 'Country', silent=True),  # silent is used to ignore exceptions
                                                         # raised while getting field
    
    # anyfield F expressions suports basic logic operations
    HField((F.sale_order_ids & 
            F.sale_order_ids.as_html_table('id',
                                           HField('as_table').with_args('_name',
                                                                        'date_order',
                                                                        'amount_total',
                                                                        'state')
            ) |
            False),
           'Sale orders'),
)  
partners_tbl

Name                                    Country         Sale orders
--------------------------------------  --------------  --------------------------------------------------------------
Administrator                                           False
Agrolait                                Belgium         <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2d3dd10>
Agrolait, Michel Fletcher               Belgium         False
Agrolait, Thomas Passot                 Belgium         False
ASUSTeK                                 Taiwan          False
ASUSTeK, Joseph Walters                 Taiwan          False
ASUSTeK, Tang Tsui                      Taiwan          False
Axelor                                  France          False
Axelor, Laith Jubair                    France          False
Bank Wealthy and sons                   United Kingdom  <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2d3dd50>
Bank Wealthy and sons, Charlie Bernard  United Kingdom  False
Bank Wealthy and sons, Jessica Dupont   United Kingdom  False
Best Designers                          India           False
Best Designers, Ayaan Agarwal           India           False
Camptocamp                              France          False
Camptocamp, Luc Maurer                  France          False
Chamber Works                           United States   <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2d3dfd0>
Chamber Works, Angel Cook               United States   False
Chamber Works, Robert Anderson          United States   False
China Export                            China           False
China Export, Chao Wang                 China           False
China Export, Zhi Ch'ang                China           False
Delta PC                                United States   False
Delta PC, Brian Williams                United States   False
Delta PC, Paul Williams                 United States   False
Delta PC, Richard Ellis                 United States   False
Demo Portal User                                        False
Demo User                                               False
Elec Import                             United States   False
Elec Import, James Miller               United States   False
Epic Technologies                       United States   False
Epic Technologies, David Simpson        United States   False
Epic Technologies, John M. Brown        United States   False
Global Solutions                        United Kingdom  False
Global Solutions, Kevin Clarke          United Kingdom  False
Global Solutions, Morgan Rose           United Kingdom  False
Global Solutions, Robin Smith           United Kingdom  False
Luminous Technologies                   Spain           <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2dca4d0>
Luminous Technologies, Laura Castro     Spain           False
Luminous Technologies, Sergio Pérez     Spain           False
Mediapole                               Germany         False
Mediapole, Phillipp Miller              Germany         False
Millennium Industries                   United Kingdom  <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2dca550>
Millennium Industries, Jacob Taylor     United Kingdom  False
Nebula Business                         Argentina       False
Nebula Business, Benjamin Flores        Argentina       False
Seagate                                 United States   False
Seagate, Edward Foster                  United States   False
Spark Systems                           Brazil          False
Spark Systems, Arthur Gomez             Brazil          False
Spark Systems, Julia Rivero             Brazil          False
Template User                                           False
The Jackson Group                       United States   False
The Jackson Group, Daniel Jackson       United States   False
The Jackson Group, William Thomas       United States   False
Think Big Systems                       United States   <openerp_proxy.ext.repr.generic.HTMLTable object at 0x2dca5d0>